In [29]:
%load_ext autoreload

%autoreload 2

In [38]:
from icd9cms import search
import os
import numpy as np
import pandas as pd
from collections import defaultdict
import json
from itertools import chain
import ipywidgets as widgets
from IPython.display import display
import re
from collections import Counter
import difflib
from differ import html_diffs, align_seqs, sentencize, \
    diff_ratio, show_diffs, compute_sequential_diff_metrics, \
    split_notes, compute_avgs
from datasets import list_metrics, load_metric

In [7]:
codes = pd.read_csv('../data/primary_codes.csv')
sorted_primary_codes_df =codes['icd9_code'].value_counts().rename_axis('code').reset_index()
sorted_primary_codes_df['long_title'] = sorted_primary_codes_df['code'].apply(lambda c: search(c).long_desc or search(c).short_desc)
sorted_primary_codes_df.columns = ['code', 'count', 'long_title']
sorted_primary_codes_df = sorted_primary_codes_df[~sorted_primary_codes_df.code.str.startswith('V3')]

In [8]:
sorted_primary_codes_df[sorted_primary_codes_df.code.str.startswith('41401')]

,code,count,long_title
1,41401,3498,Coronary atherosclerosis of native coronary ar...


In [9]:
sorted_primary_codes_df[sorted_primary_codes_df.code.str.startswith('41')].head(5)

,code,count,long_title
1,41401,3498,Coronary atherosclerosis of native coronary ar...
4,41071,1751,"Subendocardial infarction, initial episode of ..."
15,41041,482,Acute myocardial infarction of other inferior ...
16,41011,482,Acute myocardial infarction of other anterior ...
20,41519,343,Other pulmonary embolism and infarction


In [10]:
sorted_primary_codes_df[sorted_primary_codes_df.code.str.startswith('434')].long_title.tolist()

['Cerebral embolism with cerebral infarction',
 'Cerebral artery occlusion, unspecified with cerebral infarction',
 'Cerebral thrombosis with cerebral infarction',
 'Cerebral artery occlusion, unspecified without mention of cerebral infarction',
 'Cerebral embolism without mention of cerebral infarction']

### 41401 Notes

In [30]:
notes = pd.read_csv('../data/41401_notes.csv')

In [31]:
discharge_notes = notes[(notes.category == 'Discharge summary') & (notes.description != 'Addendum')]

In [33]:
care_notes_df, discharge_notes_df = split_notes(notes)

In [34]:
care_notes_df['cat_descs'] = care_notes_df.apply(lambda r: [(f'{c.strip()}:{d}') for c,d in zip(r.categories, r.descriptions)], axis=1)

In [23]:
results_df = compute_diff(stroke_notes_subset)

2021-01-19 12:49:06,517: Removing /Users/tom/.cache/huggingface/metrics/bert_score/default/default_experiment-1-0.arrow
2021-01-19 12:49:06,524: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow


In [386]:
results_df = pd.concat([rougeL, results_df.loc[:, [c for c in results_df.columns if 'rougeL' in c]]], axis=1)

In [399]:
scores = results_df.loc[:, ['category', 'description', 'avg_diff_ratio', 'rougeL_recall_med', 'rougeL_recall_iqr', 'rougeL_precision_med', 'rougeL_precision_iqr', 
                            'bert-score_recall_med', 'bert-score_recall_iqr', 'bert-score_precision_med', 'bert-score_precision_iqr']]

In [400]:
scores.sort_values(['rougeL_recall_med', 'rougeL_precision_med'], ascending=[False, True])

,category,description,avg_diff_ratio,rougeL_recall_med,rougeL_recall_iqr,rougeL_precision_med,rougeL_precision_iqr,bert-score_recall_med,bert-score_recall_iqr,bert-score_precision_med,bert-score_precision_iqr
2,Nursing,Nursing Transfer Note,0.872569,0.997854,0.000000,0.621658,0.000000,0.982755,0.000000,0.744808,0.000000
3,Nutrition,Clinical Nutrition Note,0.154612,0.921967,0.323136,0.523822,0.138380,0.916548,0.203272,0.601892,0.248073
4,Physician,Intensivist Note,0.234087,0.651907,0.127212,0.692593,0.118770,0.857477,0.060351,0.853575,0.046447
1,Nursing,Nursing Progress Note,0.440298,0.632599,0.675619,0.666618,0.682953,0.701072,0.424521,0.730515,0.501814
5,Radiology,CHEST (PORTABLE AP),0.088830,0.417902,0.021350,0.540940,0.270825,0.791767,0.009032,0.827279,0.074302
6,Radiology,CT HEAD W/O CONTRAST,0.088893,0.411212,0.231364,0.399956,0.201956,0.755250,0.107351,0.772630,0.083226
7,Social Work,Social Work Progress Note,0.073434,0.383871,0.000000,0.162568,0.000000,0.702947,0.000000,0.552549,0.000000
0,ECG,Report,0.186325,0.331579,0.167044,0.231640,0.159380,0.569639,0.144755,0.428310,0.057963


In [402]:
scores.sort_values(['bert-score_recall_med', 'bert-score_precision_med'], ascending=[False, True])

,category,description,avg_diff_ratio,rougeL_recall_med,rougeL_recall_iqr,rougeL_precision_med,rougeL_precision_iqr,bert-score_recall_med,bert-score_recall_iqr,bert-score_precision_med,bert-score_precision_iqr
2,Nursing,Nursing Transfer Note,0.872569,0.997854,0.000000,0.621658,0.000000,0.982755,0.000000,0.744808,0.000000
3,Nutrition,Clinical Nutrition Note,0.154612,0.921967,0.323136,0.523822,0.138380,0.916548,0.203272,0.601892,0.248073
4,Physician,Intensivist Note,0.234087,0.651907,0.127212,0.692593,0.118770,0.857477,0.060351,0.853575,0.046447
5,Radiology,CHEST (PORTABLE AP),0.088830,0.417902,0.021350,0.540940,0.270825,0.791767,0.009032,0.827279,0.074302
6,Radiology,CT HEAD W/O CONTRAST,0.088893,0.411212,0.231364,0.399956,0.201956,0.755250,0.107351,0.772630,0.083226
7,Social Work,Social Work Progress Note,0.073434,0.383871,0.000000,0.162568,0.000000,0.702947,0.000000,0.552549,0.000000
1,Nursing,Nursing Progress Note,0.440298,0.632599,0.675619,0.666618,0.682953,0.701072,0.424521,0.730515,0.501814
0,ECG,Report,0.186325,0.331579,0.167044,0.231640,0.159380,0.569639,0.144755,0.428310,0.057963


In [ ]:
# sort and re-rank, set of documents, extract phenotypes to test how excluding 
# the set of redundant document effects phenotype extraction??

In [ ]:
# plots between max / min... as things progress... - heavy bias between how the clinician records the notes..
# 

In [479]:
avgs = compute_avgs(more_notes_results_df)

In [495]:
avgs.loc[:, ['cat_desc', 'micro_avg', 'rg_rec_avg', 'rg_prec_avg', 'bs_rec_avg', 'bs_prec_avg']]

,cat_desc,micro_avg,rg_rec_avg,rg_prec_avg,bs_rec_avg,bs_prec_avg
0,Nursing:Nursing Progress Note,0.227994,0.345551,0.348121,0.496873,0.494258
1,Physician :Intensivist Note,0.169265,0.594225,0.597550,0.856726,0.858064
2,Radiology:CT HEAD W/O CONTRAST,0.084276,0.400569,0.471806,0.732597,0.739952
3,Nursing:Nursing Transfer Note,0.421328,0.725352,0.545839,0.701177,0.572983
4,Nutrition:Clinical Nutrition Note,0.278534,0.723751,0.591569,0.789677,0.708747


### Compare Discharge Summary to last note types via Summarisation Metrics
- recall will likely be low, but any non-zero ness is potentially interesting
- do neural methods work at all here??

In [ ]:
# only take last note...? Or take all notes of each category
#

In [502]:
rouge_metric.compute(rouge_types=['rougeL'], use_agregator=False)['rougeL']

2021-01-15 00:19:26,721: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow


[Score(precision=0.3333333333333333, recall=1.0, fmeasure=0.5),
 Score(precision=0.3333333333333333, recall=1.0, fmeasure=0.5)]

In [11]:
def calc_diff_metrics(ds: str, prev_notes: dict):
    # calc diff between 
    rouge_metric = load_metric('rouge')
    bert_metric = load_metric('bertscore')
    output = defaultdict(list)
    for k, note_text in prev_notes.items():
        ratio, std_dev, max_ratio, min_ratio, redundant_toks_len, total_toks = diff_ratio(ds, note_text)
        output['ratio'].append(ratio)
        output['cat_desc'].append(':'.join(k))
        rouge_metric.add(prediction=ds, reference=note_text)
        bert_metric.add(prediction=ds, reference=note_text)
#     bert_scores = bert_metric.compute(lang='en', rescale_with_baseline=True, model_type='xlnet-base-cased')
#     output['bs_prec'].extend(bert_scores.precision)
#     output['bs_rec'].extend(bert_scores.recall)
    rougeL = rouge_metric.compute(rouge_types=['rougeL'], use_agregator=False)['rougeL']
    output['rg_prec'].extend([s.precision for s in rougeL])
    output['rg_rec'].extend([s.recall for s in rougeL])
    return pd.DataFrame(output)
    
def calc_discharge_summary_redundancy(notes: pd.DataFrame):
    summaries = notes[notes.category == 'Discharge summary'].loc[:, ['hadm_id', 'text']]
    ds_diffs = []
    for r in summaries.itertuples():
        hadm_notes = notes[(notes.hadm_id == r.hadm_id) & (notes.category != 'Discharge summary')]
        last_notes_per_type = {}
        for k, df_g in hadm_notes.groupby(['category', 'description']):
            last_notes_per_type[':'.join(k)] = df_g.text.iloc[-1]
        ds_diffs.append(calc_diff_metrics(r.text, last_notes_per_type))
    return pd.concat(ds_diffs)

In [518]:
calc_discharge_summary_redundancy(stroke_notes_subset)

2021-01-15 00:24:18,249: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow
2021-01-15 00:24:21,168: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow


In [38]:
print('Token level / global note redundancy with concurrent notes:')
print(f'Overall Average Internote type redundnacy:{results_df.avg_diff_ratio.sum() / results_df.avg_diff_ratio.shape[0]}')

Token level / global note redundancy with concurrent notes:


NameError: name 'results_df' is not defined

### 43411 Note Analysis

In [6]:
stroke_notes = pd.read_csv('../data/43411_notes.csv')

In [14]:
stroke_care_notes_df, stroke_discharge_df = split_notes(stroke_notes)

In [11]:
comp_metrics.sort_values('rougeL_rec', ascending=False)

,hadm_id,category,description,avg_diff_ratio,rougeL_rec,rougeL_prec
247,121824,Physician,Physician Resident Admission Note,0.853598,1.000000,0.769582
98,107913,Physician,Physician Surgical Admission Note,0.996592,1.000000,0.980357
860,180112,Nursing,Nursing Transfer Note,0.932375,1.000000,0.420744
450,140761,Physician,Physician Surgical Admission Note,0.999477,1.000000,0.996753
586,154473,General,Cardiology consult,0.904114,1.000000,0.949074
...,...,...,...,...,...,...
1079,195985,Nursing/other,Report,0.035890,0.073285,0.183257
889,182153,Nursing/other,Report,0.036163,0.066773,0.135223
799,171903,Nursing/other,Report,0.028571,0.060440,0.180328
122,109832,Nursing,Nursing Transfer Note,0.037115,0.049407,0.806452


In [22]:
stroke_notes.category.unique()

array(['Nursing', 'Physician ', 'Rehab Services', 'Nutrition',
       'Respiratory ', 'Social Work', 'General', 'Radiology',
       'Nursing/other', 'Discharge summary', 'Echo', 'ECG', 'Pharmacy',
       'Case Management '], dtype=object)

In [30]:
texts = stroke_notes[(stroke_notes.hadm_id == 121824) & (stroke_notes.category == 'Physician ') & (stroke_notes.description == 'Physician Resident Admission Note')].text.tolist()

In [45]:
group_avgs = compute_avgs(results_df)

In [46]:
group_avgs['redundant_ratio'] = group_avgs['redundant_toks'] / group_avgs.total_toks

In [47]:
group_avgs.sort_values('redundant_ratio', ascending=False).reset_index(drop=True).head(20)

,cat_desc,redundant_toks,total_toks,avg_txt_len,macro_avg,micro_avg,num_instances,redundant_ratio
0,Physician :Physician Resident Admission Note,28842.0,33109,12525.819048,0.856518,0.809448,19,0.871123
1,Physician :Physician Surgical Progress Note,3181.0,4347,5587.000000,0.809635,0.764214,7,0.731769
2,Physician :Physician Attending Progress Note,14380.0,21148,5941.186706,0.532457,0.536417,31,0.679970
3,Physician :Intensivist Note,100737.0,159909,5958.690231,0.691003,0.639584,211,0.629965
4,Respiratory :Respiratory Care Shift Note,7263.0,12124,1230.545915,0.404803,0.443782,79,0.599060
5,Nursing:Nursing Transfer Note,30799.0,53260,2230.841242,0.598621,0.550831,159,0.578276
6,Physician :Physician Resident Progress Note,40675.0,73517,6890.886364,0.551072,0.467104,94,0.553273
7,Nursing:Nursing Progress Note,64447.0,136297,1291.039215,0.442277,0.419754,620,0.472842
8,Radiology:PORTABLE ABDOMEN,923.0,2992,1084.420513,0.274080,0.259291,23,0.308489
9,Radiology:CHEST PORT. LINE PLACEMENT,1237.0,4903,1240.143860,0.203611,0.219498,34,0.252295


In [91]:
pt = stroke_notes[stroke_notes.hadm_id == 195402]

2021-01-07 10:30:29,573: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow


In [100]:
pt_nurs_notes = pt[pt.description == 'Nursing Progress Note']

In [125]:
def single_group_ratios(pt):
    internote_sort_cols = ['hadm_id', 'chartdate', 'charttime', 'category', 'description']
    internote_group_keys = ['hadm_id', 'category', 'description']
    groups_gen = pt.sort_values(internote_sort_cols).groupby(internote_group_keys)
    results = defaultdict(list)
    metric = load_metric('rouge')
    score = None
    ratios = []
    for k, g in [list(groups_gen)[-1]]:
        if g.shape[0] > 1:
            for i in range(len(g.text.tolist()) - 1):
                metric.add(prediction=g.text.iloc[i], reference=g.text.iloc[i+1])
                ratio, std_dev, max_ratio, min_ratio, redundant_toks_len, total_toks = \
                    diff_ratio(g.text.iloc[i], g.text.iloc[i+1])
                ratios.append(ratio)
            score = metric.compute(rouge_types=['rougeL'])
    return ratios, score

In [126]:
ratios, rouge_scores = single_group_ratios(pt_nurs_notes)

2021-01-07 11:32:03,220: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow


In [48]:
discharge_notes_df = stroke_discharge_df

In [94]:
g.text

43    SW consulted to participate in family/team mee...
34    Team/family meeting to confer on pt\ns son and...
Name: text, dtype: object

### Brief Hospital Course Analysis

In [14]:
# ptn = re.compile(r'(?<=brief\shospi§tal\scourse\:)(?:.*\n)+?(?=\n\n)', flags=re.I)
ptn = re.compile(r'(?<=ospital\scourse\:)(?:.*\n)+?(?=\n\n)', flags=re.I)
def extract_hospital_course(row):
    m = re.search(ptn, row.text)
    if m:
        return re.split(r'(\n\n.*\:\n?)', row.text[m.start():m.end()])[0].strip()
    return None 

In [27]:
pmx_ptn = re.compile(r'(?:past\smedical\shistory\:?)\n?(?:.*\n)+(?=\n)')
def extract_pmx(row):
    m = re.search(pmx_ptn, row.text)
    if m:
        return re.split(r'(\n\n.*\:\n?)', row.text[m.start():m.end()])[0].strip()
    return None

In [15]:
discharge_notes_df['hospital_course_text'] = discharge_notes_df.apply(extract_hospital_course, axis=1)

In [28]:
discharge_notes_df['pmx_text'] = discharge_notes_df.apply(extract_hospital_course, axis=1)

In [25]:
discharge_notes_df[pd.isna(discharge_notes_df.pmx_text)].shape

(262, 4)

In [ ]:
print(discharge_notes_df[pd.isna(discharge_notes_df.pmx_text)].text.iloc[1])

In [150]:
metric = load_metric('bertscore')

In [ ]:
metric

In [151]:
metric.add(prediction=nut_notes.iloc[0], reference=nut_notes.iloc[1])

In [152]:
metric.compute(lang='en', rescale_with_baseline=True, model_type='xlnet-base-cased')

2021-01-13 12:30:53,610: Lock 140536987522512 acquired on /Users/tom/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock



2021-01-13 12:30:53,940: Lock 140536987522512 released on /Users/tom/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock
2021-01-13 12:30:54,218: Lock 140537333938576 acquired on /Users/tom/.cache/huggingface/transformers/df73bc9f8d13bf2ea4dab95624895e45a550a0f0a825e41fc25440bf367ee3c8.d93497120e3a865e2970f26abdf7bf375896f97fde8b874b70909592a6c785c9.lock



2021-01-13 12:30:56,199: Lock 140537333938576 released on /Users/tom/.cache/huggingface/transformers/df73bc9f8d13bf2ea4dab95624895e45a550a0f0a825e41fc25440bf367ee3c8.d93497120e3a865e2970f26abdf7bf375896f97fde8b874b70909592a6c785c9.lock
2021-01-13 12:30:56,821: Lock 140534691015632 acquired on /Users/tom/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987.lock



2021-01-13 12:32:41,918: Lock 140534691015632 released on /Users/tom/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987.lock
2021-01-13 12:32:51,689: Removing /Users/tom/.cache/huggingface/metrics/bert_score/default/default_experiment-1b50ecca-ea05-4abe-b070-60cd6a8343b4-1-0.arrow


{'precision': tensor([0.9828]),
 'recall': tensor([0.7448]),
 'f1': tensor([0.8541]),
 'hashcode': 'xlnet-base-cased_L5_no-idf_version=0.3.7(hug_trans=4.1.1)-rescaled'}

In [134]:
show_diffs(nut_notes.iloc[0], nut_notes.iloc[1])